# Text-to-Speech synthesis using OuteTTS and OpenVINO

<div class="alert alert-block alert-danger"> <b>Important note:</b> This notebook requires python >= 3.10. Please make sure that your environment fulfill to this requirement before running it </div>

[OuteTTS-0.1-350M](https://huggingface.co/OuteAI/OuteTTS-0.1-350M) is a novel text-to-speech synthesis model that leverages pure language modeling without external adapters or complex architectures, built upon the LLaMa architecture. It demonstrates that high-quality speech synthesis is achievable through a straightforward approach using crafted prompts and audio tokens.

More details about model can be found in [original repo](https://github.com/edwko/OuteTTS).

In this tutorial we consider how to run OuteTTS pipeline using OpenVINO.

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Convert model](#Convert-model)
- [Run model inference](#Run-model-inference)
    - [Text-to-Speech generation](#Text-to-Speech-generation)
    - [Text-to-Speech generation with Voice Cloning](#Text-to-Speech-generation-with-Voice-Cloning)
- [Quantization](#Quantization)
    - [Prepare calibration dataset](#Prepare-calibration-dataset)
    - [Quantize model](#Quantize-model)
    - [Verifying quantized model execution](#Verifying-quantized-model-execution)
    - [Comparing original and quantized model performance](#Comparing-model-performance)
- [Interactive demo](#Interactive-demo)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/outetts-text-to-speech/outetts-text-to-speech.ipynb" />


## Prerequisites
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import requests
from pathlib import Path

utility_files = ["skip_kernel_extension.py", "cmd_helper.py", "notebook_utils.py", "pip_helper.py"]
base_utility_url = "https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/"

for utility_file in utility_files:
    if not Path(utility_file).exists():
        r = requests.get(base_utility_url + utility_file)
        with Path(utility_file).open("w") as f:
            f.write(r.text)


helper_files = ["gradio_helper.py", "ov_outetts_helper.py"]
base_helper_url = "https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/outetts-text-to-speech"

for helper_file in helper_files:
    if not Path(helper_file).exists():
        r = requests.get(base_helper_url + helper_file)
        with Path(helper_file).open("w") as f:
            f.write(r.text)

%load_ext skip_kernel_extension

In [ ]:
import platform
from pip_helper import pip_install

pip_install(
    "-q",
    "torch>=2.1",
    "torchaudio",
    "einops",
    "transformers>=4.46.1",
    "loguru",
    "inflect",
    "pesq",
    "torchcrepe",
    "natsort",
    "polars",
    "uroman",
    "mecab-python3",
    "unidic-lite",
    "--extra-index-url",
    "https://download.pytorch.org/whl/cpu",
)
pip_install(
    "-q",
    "gradio>=4.19",
    "openvino>=2024.4.0",
    "tqdm",
    "pyyaml",
    "librosa",
    "soundfile",
    "nncf",
)
pip_install("-q", "git+https://github.com/huggingface/optimum-intel.git", "--extra-index-url", "https://download.pytorch.org/whl/cpu")

if platform.system() == "Darwin":
    pip_install("-q", "numpy<2.0.0")

In [ ]:
# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from notebook_utils import collect_telemetry

collect_telemetry("outetts-text-to-speech.ipynb")

In [ ]:
from cmd_helper import clone_repo

repo_path = clone_repo("https://github.com/edwko/OuteTTS.git")

interface_path = repo_path / "outetts/version/v1/interface.py"

updated_version = interface_path.exists()

if not updated_version:
    interface_pth = repo_path / "outetts/v0_1/interface.py"
orig_interface_path = interface_path.parent / "_orig_interface.py"

if not updated_version and not orig_interface_path.exists():
    interface_path.rename(orig_interface_path)
    # sounddevice requires to install manually additional libraries, as we do not plan to use it for audio playing
    # move it closer to its usage for avoid errors
    with orig_interface_path.open("r") as in_file:
        content = in_file.read()
        upd_content = content.replace("import sounddevice as sd", "")
        upd_content = upd_content.replace("sd.play", "import sounddevice as sd\n        sd.play")
    with interface_path.open("w") as out_file:
        out_file.write(upd_content)

%pip install -q {repo_path} --extra-index-url https://download.pytorch.org/whl/cpu

## Convert model
[back to top ⬆️](#Table-of-contents:)


 OpenVINO supports PyTorch models via conversion to OpenVINO Intermediate Representation format.  For convenience, we will use OpenVINO integration with HuggingFace Optimum. 🤗 [Optimum Intel](https://huggingface.co/docs/optimum/intel/index) is the interface between the 🤗 Transformers and Diffusers libraries and the different tools and libraries provided by Intel to accelerate end-to-end pipelines on Intel architectures.

Among other use cases, Optimum Intel provides a simple interface to optimize your Transformers and Diffusers models, convert them to the OpenVINO Intermediate Representation (IR) format and run inference using OpenVINO Runtime. `optimum-cli` provides command line interface for model conversion and optimization. 

General command format:

```bash
optimum-cli export openvino --model <model_id_or_path> --task <task> <output_dir>
```

where task is task to export the model for, if not specified, the task will be auto-inferred based on the model. You can find a mapping between tasks and model classes in Optimum TaskManager [documentation](https://huggingface.co/docs/optimum/exporters/task_manager). Additionally, you can specify weights compression using `--weight-format` argument with one of following options: `fp32`, `fp16`, `int8` and `int4`. Fro int8 and int4 [nncf](https://github.com/openvinotoolkit/nncf) will be used for  weight compression. More details about model export provided in [Optimum Intel documentation](https://huggingface.co/docs/optimum/intel/openvino/export#export-your-model).

As OuteTTS utilizes pure language modeling approach, model conversion process remains the same like conversion LLaMa models family for text generation purposes.

In [ ]:
from cmd_helper import optimum_cli
from pathlib import Path

model_id = "OuteAI/OuteTTS-0.1-350M"
model_dir = Path(model_id.split("/")[-1] + "-ov")

if not model_dir.exists():
    optimum_cli(model_id, model_dir, additional_args={"task": "text-generation-with-past"})

## Run model inference
[back to top ⬆️](#Table-of-contents:)


OpenVINO integration with Optimum Intel provides ready-to-use API for model inference that can be used for smooth integration with transformers-based solutions. For loading model, we will use `OVModelForCausalLM` class that have compatible interface with Transformers LLaMa implementation. For loading a model, `from_pretrained` method should be used. It accepts path to the model directory or model_id from HuggingFace hub (if model is not converted to OpenVINO format, conversion will be triggered automatically). Additionally, we can provide an inference device, quantization config (if model has not been quantized yet) and device-specific OpenVINO Runtime configuration. More details about model inference with Optimum Intel can be found in [documentation](https://huggingface.co/docs/optimum/intel/openvino/inference). We will use `OVModelForCausalLM` as replacement of original `AutoModelForCausalLM` in `InterfaceHF`.


In [ ]:
from notebook_utils import device_widget

device = device_widget(exclude=["NPU"])

device

In [ ]:
from ov_outetts_helper import InterfaceOV, OVHFModel  # noqa: F401

# Uncomment these lines to see pipeline details
# ??InterfaceOV
# ??OVHFModel

In [ ]:
interface = InterfaceOV(model_dir, device.value)

### Text-to-Speech generation
[back to top ⬆️](#Table-of-contents:)


Now let's see model in action. Providing input text to `generate` method of interface, model returns tensor that represents output audio with random speaker characteristics.

In [ ]:
tts_output = interface.generate(text="Hello, I'm working!", temperature=0.1, repetition_penalty=1.1, max_length=4096)

In [ ]:
import IPython.display as ipd


def play(data, rate=None):
    ipd.display(ipd.Audio(data, rate=rate))


play(tts_output.audio[0].numpy(), rate=tts_output.sr)

### Text-to-Speech generation with Voice Cloning
[back to top ⬆️](#Table-of-contents:)


Additionally, we can specify reference voice for generation by providing reference audio and transcript for it. `interface.create_speaker` processes reference audio and text to set of features used for audio description.

In [ ]:
from notebook_utils import download_file

ref_audio_url = "https://huggingface.co/OuteAI/OuteTTS-0.1-350M/resolve/main/samples/2.wav"
file_path = Path("2.wav")

if not file_path.exists():
    file_path = download_file(ref_audio_url)

In [ ]:
play(file_path)

In [ ]:
speaker = interface.create_speaker(file_path, "Hello, I can speak pretty well, but sometimes I make some mistakes.")

# Save the speaker to a file
interface.save_speaker(speaker, "speaker.pkl")

# Load the speaker from a file
speaker = interface.load_speaker("speaker.pkl")

# Generate TTS with the custom voice
cloned_output = interface.generate(
    text="This is a cloned voice speaking",
    speaker=speaker,
    temperature=0.1,
    repetition_penalty=1.1,
    max_length=4096,
)

In [ ]:
play(cloned_output.audio[0].numpy(), rate=cloned_output.sr)

## Quantization
[back to top ⬆️](#Table-of-contents:)


[NNCF](https://github.com/openvinotoolkit/nncf/) enables post-training quantization by adding the quantization layers into the model graph and then using a subset of the training dataset to initialize the parameters of these additional quantization layers. The framework is designed so that modifications to your original training code are minor.

The optimization process contains the following steps:

1. Create a calibration dataset for quantization.
2. Run `nncf.quantize` to obtain quantized model.
3. Serialize the `INT8` model.

Note: Quantization is a time and memory-consuming operation. Running the quantization code below may take some time.

In [ ]:
from notebook_utils import quantization_widget

to_quantize = quantization_widget()

to_quantize

### Prepare calibration dataset
[back to top ⬆️](#Table-of-contents:)

The first step is to prepare calibration datasets for quantization. We will utilize [Filtered LibriTTS-R](https://huggingface.co/datasets/parler-tts/libritts_r_filtered) dataset as it was used to train the original model.

In [ ]:
%%skip not $to_quantize.value

from datasets import load_dataset

libritts = load_dataset("parler-tts/libritts_r_filtered", "clean", split="test.clean", streaming=True)

### Quantize model
[back to top ⬆️](#Table-of-contents:)

Below we run the quantize function which calls `nncf.quantize` on the OpenVINO IR model and collected dataset.

In [ ]:
%%skip not $to_quantize.value

import nncf
from functools import partial
import numpy as np

def transform_fn(item, interface):
    text_normalized = item["text_normalized"]
    prompt = interface.prompt_processor.get_completion_prompt(text_normalized, interface.language, None)
    encoded = interface.prompt_processor.tokenizer(prompt, return_tensors="np")

    input_ids = encoded["input_ids"]
    attention_mask = encoded["attention_mask"]
    inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

    position_ids = np.cumsum(attention_mask, axis=1) - 1
    position_ids[attention_mask == 0] = 1
    inputs["position_ids"] = position_ids

    batch_size = input_ids.shape[0]
    inputs["beam_idx"] = np.arange(batch_size, dtype=int)

    return inputs

hf_model = OVHFModel(model_dir, device.value).model
dataset = nncf.Dataset(libritts, partial(transform_fn, interface=interface))

quantized_model = nncf.quantize(
    hf_model.model,
    dataset,
    preset=nncf.QuantizationPreset.MIXED,
    model_type=nncf.ModelType.TRANSFORMER,
    ignored_scope=nncf.IgnoredScope(
        patterns=[
            # We need to use ignored scope for this pattern to generate the most efficient model
            "__module.model.layers.*.self_attn/aten::scaled_dot_product_attention/ScaledDotProductAttention"
        ]
    )
)

hf_model.model = quantized_model
model_dir_quantized = Path(f"{model_dir}_quantized")
hf_model.save_pretrained(model_dir_quantized)
interface.prompt_processor.tokenizer.save_pretrained(model_dir_quantized)

### Verifying quantized model execution
[back to top ⬆️](#Table-of-contents:)

In order to verify the quality of the quantized model, we will generate outputs based on the same texts and speakers used for the non-quantized model.
First, we will save the quantized model and recreate the pipelines for validation.
Then we will generate the outputs and try to compare them with the previously obtained outputs.

In [ ]:
%%skip not $to_quantize.value

interface_quantized = InterfaceOV(model_dir_quantized, device.value)

In [ ]:
%%skip not $to_quantize.value

tts_output_quantized = interface_quantized.generate(text="Hello, I'm working!", temperature=0.1, repetition_penalty=1.1, max_length=4096)

In [ ]:
%%skip not $to_quantize.value
# Non-quantized model output:
play(tts_output.audio[0].numpy(), rate=tts_output.sr)

In [ ]:
%%skip not $to_quantize.value
# Quantized model output:
play(tts_output_quantized.audio[0].numpy(), rate=tts_output_quantized.sr)

In [ ]:
%%skip not $to_quantize.value

speaker_quantized = interface_quantized.load_speaker("speaker.pkl")
cloned_output_quantized = interface_quantized.generate(
    text="This is a cloned voice speaking",
    speaker=speaker,
    temperature=0.1,
    repetition_penalty=1.1,
    max_length=4096,
)

In [ ]:
%%skip not $to_quantize.value
# Non-quantized model output:
play(cloned_output.audio[0].numpy(), rate=cloned_output.sr)

In [ ]:
%%skip not $to_quantize.value
# Quantized model output:
play(cloned_output_quantized.audio[0].numpy(), rate=cloned_output_quantized.sr)

### Comparing model performance
[back to top ⬆️](#Table-of-contents:)

In [ ]:
%%skip not $to_quantize.value

import time
import tqdm

def calculate_inference_time(interface, dataset, limit):
    inference_time = []
    for i, item in tqdm.tqdm(enumerate(dataset), total=limit):
        if i > limit: break
        start = time.perf_counter()
        _ = interface.generate(
            text=item["text_normalized"],
            max_length=256,
            additional_gen_config={
                "pad_token_id": interface.prompt_processor.tokenizer.eos_token_id
            }
        )
        end = time.perf_counter()
        delta = end - start
        inference_time.append(delta)
    return np.median(inference_time)

interface = InterfaceOV(model_dir, device.value)
limit = 25

fp_inference_time = calculate_inference_time(interface, libritts, limit)
print(f"Original model generate time: {fp_inference_time}")

interface_quantized = InterfaceOV(model_dir_quantized, device.value)
int_inference_time = calculate_inference_time(interface_quantized, libritts, limit)
print(f"Quantized model generate time: {int_inference_time}")

## Interactive demo
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import ipywidgets as widgets

quantized_model_present = model_dir_quantized.exists()

use_quantized_model = widgets.Checkbox(
    value=True if quantized_model_present else False,
    description="Use quantized model",
    disabled=False,
)

use_quantized_model

In [ ]:
from gradio_helper import make_demo

if use_quantized_model:
    demo_interface = InterfaceOV(model_dir_quantized, device.value)
else:
    demo_interface = InterfaceOV(model_dir, device.value)

demo = make_demo(demo_interface)

try:
    demo.launch(debug=True)
except Exception:
    demo.launch(share=True, debug=True)